In [1]:
import tensorflow as tf
import time

In [2]:
people_per_batch = tf.constant(6000)
_nrof_images_per_class = [5 for i in xrange(6000)]
nrof_images_per_class = tf.constant(_nrof_images_per_class)
emb_start_idx = tf.Variable(0)
triplets = tf.constant([-1,-1,-1,-1,-1,-1], shape=[6,1], name='triplets')
pair = tf.Variable(1)
a = tf.constant(0)
def cond_a(a, emb_start_idx, triplets):
    r = tf.less(a, people_per_batch)
    return r 

def body_a(a, emb_start_idx, triplets):
    b = tf.constant(1)
    def cond_b(a,b,emb_start_idx,triplets):
        return tf.less(b, nrof_images_per_class[a])
    
    def body_b(a, b, emb_start_idx, triplets):       
        pair = b
        def cond_pair(a, b, pair, emb_start_idx, triplets):
            return tf.less(pair, nrof_images_per_class[a])
        
        def body_pair(a, b, pair, emb_start_idx, triplets):
            a_idx = tf.add(emb_start_idx, tf.subtract(b,1))
            p_idx = tf.add(emb_start_idx, pair)
            
            ####
            
            triplet = tf.stack([a, b, pair, emb_start_idx, a_idx, p_idx], name='triplet')
            triplet = tf.reshape(triplet, shape=(6,1))
            triplets = tf.concat([triplets, triplet], axis=1)
            return a, b, pair+1, emb_start_idx, triplets
        
        a, b, pair, emb_start_idx, triplets = tf.while_loop(cond_pair, body_pair, 
                                                             loop_vars=[a, b, pair, emb_start_idx, triplets],
                                                             shape_invariants=[a.shape, 
                                                                               b.shape, 
                                                                               pair.shape, 
                                                                               emb_start_idx.shape, 
                                                                               tf.TensorShape([6,None])])
        return a, b+1, emb_start_idx, triplets

    a, b, emb_start_idx, triplets = tf.while_loop(cond_b, body_b, loop_vars=[a, b, emb_start_idx, triplets],
                                                shape_invariants=[a.shape, b.shape, emb_start_idx.shape,
                                                                  tf.TensorShape([6,None])])
    
    return a+1, emb_start_idx + nrof_images_per_class[a], triplets

a, emb_start_idx, triplets = tf.while_loop(cond_a, body_a, loop_vars=[a, emb_start_idx, triplets],
                                           shape_invariants =[a.shape, emb_start_idx.shape, tf.TensorShape([6,None])])


In [3]:
sess = tf.InteractiveSession()
init = tf.global_variables_initializer()
sess.run(init)

In [4]:
start_time = time.time()
triplets = sess.run(triplets)
end_time = time.time()
print(triplets.T)
print('run time: %ds' %(end_time-start_time))

[[   -1    -1    -1    -1    -1    -1]
 [    0     1     1     0     0     1]
 [    0     1     2     0     0     2]
 ..., 
 [ 5999     3     3 29995 29997 29998]
 [ 5999     3     4 29995 29997 29999]
 [ 5999     4     4 29995 29998 29999]]
run time: 13s
